---
layout: post
title: API do IBGE
subtitle: Exercícios e Referências
tags: [python, pycharm, jupyter, package, ibge, api]
image: /img/posts/pandas_icon.png
bigimg: /img/posts/pandas_big.png
gh-repo: michelmetran/api_ibge
gh-badge: [follow, star, watch, fork]
comments: true

---

# SABESP

In [32]:
import os
import time
from selenium import webdriver
from bs4 import BeautifulSoup

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options

In [33]:
# Escolhe o driver Firefox com Profile e Options

driver = webdriver.FirefoxProfile()
driver.set_preference('intl.accept_languages', 'pt-BR, pt')
#driver.set_preference('browser.download.folderList', '2')
#driver.set_preference('browser.download.manager.showWhenStarting', 'false')
#driver.set_preference('browser.download.dir', 'dwnd_path')
#driver.set_preference('browser.helperApps.neverAsk.saveToDisk', 'application/octet-stream,application/vnd.ms-excel')

options = Options()
options.headless = False

driver = webdriver.Firefox(firefox_profile=driver, options=options)

In [34]:
# Cria um driver

site = 'http://mananciais.sabesp.com.br/HistoricoSistemas'
driver.get(site)

## Escolhe o Sistema

In [19]:
#driver.find_element_by_xpath('//*[@id="cmbSistema"]').click()

In [20]:
sistema = {0: 'Cantareira',
           1: 'Alto Tietê',
           2: 'Guarapiranga',
           3: 'Cotia',
           4: 'Rio Grande',
           5: 'Rio Claro',
           6: 'São Lourenço',}

In [21]:
#driver.find_element_by_xpath('//*[@id="cmbSistema"]').send_keys(sistema[0])
#driver.find_element_by_xpath('//*[@id="cmbSistema"]').send_keys(sistema[0])
# ToDo: Check URL

## Cria o Objeto Soup

In [22]:
WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.ID,"contenttabledivjqxGrid")))

soup = BeautifulSoup(driver.page_source, 'html.parser')

In [23]:
print(soup.prettify())

<html>
 <head>
  <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
  <meta content="" name="description"/>
  <meta content="" name="author"/>
  <meta content="IE=9;IE=11" http-equiv="X-UA-Compatible"/>
  <link href="favicon.ico" rel="shortcut icon"/>
  <link href="/css/SSD3Publico.css" rel="stylesheet"/>
  <link href="pikaday/pikaday.css" rel="stylesheet"/>
  <!-- Bootstrap core JavaScript
    ================================================== -->
  <link href="bootstrap/css/bootstrap.min.css" rel="stylesheet"/>
  <link href="font-awesome/css/font-awesome.min.css" rel="stylesheet"/>
  <link href="swal2/sweetalert2.min.css" rel="stylesheet"/>
  <script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript">
  </script>
  <script src="/js/jquery.min.js">
  </script>
  <!-- Bootstrap core JavaScript
    ================================================== -->
  <script src="bootstrap/js/bootstrap.min.js">
  </script>
  <scri

## Vê e Escolhe a Data

## Cabeçalho

In [24]:
# Cabeçalho
header = soup.find_all('div', {'class': 'jqx-grid-column-header'})
for i in header:
    print(i.get_text())

Data
Nível (m)
Volume (hm³)
Volume (%)
Q Jusante (m³/s)
Q Natural (m³/s)
Chuva (mm)
Nível (m)
Volume (hm³)
Volume (%)
Q Jusante (m³/s)
Q Natural (m³/s)
Chuva (mm)
Nível (m)
Volume (hm³)
Represa Jaguari/Jacareí
Represa Cachoeira
Represa Atibainha
Represa Paiva Castro
Represa Águas Claras
Represa Jaguari (Cesp)
Equivalente (Jaguari/Jacareí/Cachoeira/Atibainha)


In [25]:
# Seleciona as relevantes
head = []
for i in header:
    if i.get_text().startswith(('Represa', 'Equivalente')):
        print('Excluído: ' + i.get_text())
    else:
        print(i.get_text())
        head.append(i.get_text())
    
print('-'*70)
print(head)
print('-'*70)
print('Número de Colunas: ' + str(len(head)))

Data
Nível (m)
Volume (hm³)
Volume (%)
Q Jusante (m³/s)
Q Natural (m³/s)
Chuva (mm)
Nível (m)
Volume (hm³)
Volume (%)
Q Jusante (m³/s)
Q Natural (m³/s)
Chuva (mm)
Nível (m)
Volume (hm³)
Excluído: Represa Jaguari/Jacareí
Excluído: Represa Cachoeira
Excluído: Represa Atibainha
Excluído: Represa Paiva Castro
Excluído: Represa Águas Claras
Excluído: Represa Jaguari (Cesp)
Excluído: Equivalente (Jaguari/Jacareí/Cachoeira/Atibainha)
----------------------------------------------------------------------
['Data', 'Nível (m)', 'Volume (hm³)', 'Volume (%)', 'Q Jusante (m³/s)', 'Q Natural (m³/s)', 'Chuva (mm)', 'Nível (m)', 'Volume (hm³)', 'Volume (%)', 'Q Jusante (m³/s)', 'Q Natural (m³/s)', 'Chuva (mm)', 'Nível (m)', 'Volume (hm³)']
----------------------------------------------------------------------
Número de Colunas: 15


## Valores

In [26]:
# Valores
data = soup.find_all('div', {'class': 'jqx-grid-cell'})
values = []
for i in data:
    #print(i.get_text())
    values.append(i.get_text())
    
#print(values)

## Monta Tabela

In [27]:
import numpy as np
import pandas as pd

# Convert data to numpy array
num = np.array(values)

# Currently its shape is single dimensional
n_rows = int(len(num)/len(head))
n_cols = int(len(head))
reshaped = num.reshape(n_rows, n_cols)

# Construct Table
pd.DataFrame(reshaped, columns=head)

,Data,Nível (m),Volume (hm³),Volume (%),Q Jusante (m³/s),Q Natural (m³/s),Chuva (mm),Nível (m),Volume (hm³),Volume (%),Q Jusante (m³/s),Q Natural (m³/s),Chuva (mm),Nível (m),Volume (hm³)
0,17/06/2020,"837,15","497,70","61,59","0,50","6,43","0,00","816,47","27,86","40,01","5,00","2,11","0,00","783,74","34,80"
1,16/06/2020,"837,18","499,09","61,76","0,50","5,97","0,00","816,40","27,40","39,34","4,51","1,83","0,00","783,76","35,19"
2,15/06/2020,"837,22","500,56","61,95","0,50","9,29","0,00","816,32","26,87","38,58","4,50","1,91","0,00","783,78","35,58"
3,14/06/2020,"837,25","501,78","62,10","0,50","8,83","0,00","816,23","26,27","37,72","4,50","2,05","0,00","783,80","35,97"
4,13/06/2020,"837,28","503,10","62,26","0,50","6,47","0,20","816,13","25,62","36,78","4,50","1,57","0,20","783,82","36,36"
5,12/06/2020,"837,32","504,65","62,45","0,50","10,25","0,00","816,03","24,97","35,84","3,92","2,89","0,20","783,84","36,75"
6,11/06/2020,"837,35","505,88","62,61","0,50","13,09","0,20","815,90","24,12","34,63","3,00","3,45","0,20","783,86","37,14"
7,10/06/2020,"837,37","506,87","62,73","0,50","12,13","2,00","815,79","23,41","33,62","3,00","4,60","2,40","783,84","36,75"
8,09/06/2020,"837,40","507,94","62,86","0,50","10,21","0,80","815,73","23,03","33,07","3,00","5,86","0,20","783,78","35,58"
9,08/06/2020,"837,43","509,17","63,01","0,65","15,12","0,00","815,65","22,52","32,33","3,45","7,58","0,40","783,75","35,00"


## Vê e Escolhe a Data

In [29]:
# Encontra a Data Selecionada

data = soup.find('span', id='lblDataSelecionada')
data_sel = data.text
data_sel

'18/05/2020 '

In [30]:
# Seleciona o Campo para escolher uma nova data

from selenium.webdriver.common.by import By

senddata = driver.find_element_by_xpath('//span[@id="lblDataSelecionada"]')
senddata
senddata.click()

# Return X months
for i in range(4):
    driver.find_element(By.XPATH, '//button[@class="pika-prev"]').click()
    
driver.find_element(By.XPATH, '//button[@class="pika-button pika-day"]').click()

## Encerra o Navegador

In [31]:
driver.quit()